In [ ]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hdf5storage

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, Normalizer

In [3]:
# Load the data
path="D:/alon_/Research/POSTGRAD/PhD/CODE/Adrian's Code/NETWORK_sims_2/Saved Networks/Simulations Only/Python Data/"
filename = path+'Training Simulation_1_ForPython.mat'
# filename =  path + 'Net_Sx_20_NoW500_03_30-2019_11_16_59_SimsOnly_2Electrodes_TRAINING_SIM_15-May-2019.mat'
data = loadmat(filename)

IDrain1 = data['IDrain1'].todense()
IDrain2 = data['IDrain2'].todense()
VSource1 = data['VSource1']
VSource2 = data['VSource2']

In [ ]:
# Explore
fig, axes = plt.subplots(1, 2, figsize=(16, 4))

axes[0].plot(VSource1, label = 'VSource1'); axes[0].plot(VSource2, label = 'VSource2')
axes[0].set_title("Input to device - Target for learning (y)"); axes[0].set_xlabel('Time'); axes[0].set_ylabel('V')
axes[0].legend()
axes[1].plot(IDrain1, label = 'IDrain1'); axes[1].plot(IDrain2, label = 'IDrain2')
axes[1].set_title("Output of device - Features for learning (X)"); axes[1].set_xlabel('Time'); axes[1].set_ylabel('A')
axes[1].legend()

In [ ]:
# Split the data
start_time = 0
X = np.hstack([IDrain1, IDrain2])[start_time:]
y = VSource1[start_time:].ravel()
y = (y == y.max())
y2 = VSource2[start_time:].ravel()
y2 = (y2 == y2.max()) * 2
y = y + y2

# scalar = StandardScaler()
# scalar = Normalizer()
# X = scalar.fit_transform(X)
# X = (X - X.mean(axis = 0)) / X.std(axis = 0)
X = (X - X.mean(axis = 0)) / X.std(axis = 0)
# X = (X - X.mean(axis = 1)) / X.std(axis = 1)
X = np.asarray(X) #* 10000

# sns.kdeplot(X[:,0][y], X[:,1][y])
# sns.kdeplot(X[:,0][~y], X[:,1][~y])

plt.scatter(X[:,0], X[:,1], c = y, marker = 'x', cmap = 'Set1', alpha = 0.1)

In [ ]:
# Training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0, random_state = 0)

clf = SVC(kernel = 'linear', C = 0.5, probability = True).fit(X_train, y_train)
# clf2 = LinearDiscriminantAnalysis().fit(X_train, y_train)

# y_pred = clf.predict(X_test)

# print('Test accuracy:', clf.score(X_test, y_test))

In [ ]:
# y_pred

In [ ]:
# clf.predict_proba(X_test)

In [ ]:
# Plot
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [ ]:
X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)

fig, ax = plt.subplots()
plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')

In [5]:
# Load the test data
filename = path+'Testing Simulation_2_ForPython.mat' #Change this number
data_test = loadmat(filename)

In [6]:
X_test = np.hstack([data_test['IDrain1'].todense(), data_test['IDrain2'].todense()])[start_time:]
# X_test = scalar.transform(X_test)
X_test = (X_test - X_test.mean(axis = 0)) / X_test.std(axis = 0)
X_test = (X_test - X_test.mean(axis = 1)) / X_test.std(axis = 1)
X_test = np.asarray(X_test)

y = data_test['VSource1'][start_time:].ravel()
y = (y == y.max())
y2 = data_test['VSource2'][start_time:].ravel()
y2 = (y2 == y2.max()) * 2
y_test = y + y2

y_pred = clf.predict(X_test)

NameError: name 'start_time' is not defined

In [ ]:
X0, X1 = X_test[:, 0], X_test[:, 1]
xx, yy = make_meshgrid(X0, X1)

fig, ax = plt.subplots()
plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X0, X1, c=y_test, cmap=plt.cm.coolwarm, s=20, edgecolors='k')

In [ ]:
# Explore
fig, axes = plt.subplots(1, 2, figsize=(16, 4))

axes[0].plot(data_test['VSource1'], label = 'VSource1'); axes[0].plot(data_test['VSource2'], label = 'VSource2')
axes[0].set_title("Input to device - Target for learning (y)"); axes[0].set_xlabel('Time'); axes[0].set_ylabel('V')
axes[0].legend()
axes[1].plot(data_test['IDrain1'].todense(), label = 'IDrain1'); axes[1].plot(data_test['IDrain2'].todense(), label = 'IDrain2')
axes[1].set_title("Output of device - Features for learning (X)"); axes[1].set_xlabel('Time'); axes[1].set_ylabel('A')
axes[1].legend()

In [ ]:
plt.plot(y_pred == 1, label = 'VSource1')
plt.plot(y_pred == 2, label = 'VSource2')
plt.legend()